# Torkan 2
Denna uppgiften är en fortsättningsuppgift på [torkan](https://github.com/lunduniversity/schoolprog-satellite/tree/master/exercises/drought). Vi rekommenderar att du gör den först. 

I denna uppgift ska vi undersöka hur man nogrannare kan undersöka om det är torka. Detta kommer vi göra med bland annat hjälp av NDVI som du känner igen från torkan men vi kommer också introducera NDWI (Normalized Difference Water Index) som istället för vegetation kollar på vatteninnehåll. Likt NDVI ger inte NDWI själv någon bild av hur det ser ur jämfört med tidigare år. Detta är varför vi även ska kolla på VCI (vegetation condition index). VCI används för att jämföra ett år med andra för att se hur högt eller lågt NDVI/NDWI är relativt de andra åren. Vi kommer ha med korta teoriavsnitt om både VCI och NDWI. 

## 1. Ladda in datan från drive
Innan vi kan börja analysera data måste vi ladda in data. Kör följande block för att installera och importera biblioteken du behöver:

In [0]:
!pip3 install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials  

Vi kommer ladda ner datan från google drive. För att göra detta måste du logga in på ditt googlekonto genom att klicka på länken som visas när du kör koden nedan. Detta kommer ge dig en verifikationskod som du ska klistra in i fältet som genereras av samma kod.

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Följande block laddar in och packar upp datan. Du kan se i rutan till vänster om koden vilka filer du har. 

In [0]:
download = drive.CreateFile({'id': '1PsNbvr9Kch4UtQXsohtXViJo07VSg-N9'})
download.GetContentFile('sentinel2_sw_scania.tar.gz')

import tarfile
tf = tarfile.open('sentinel2_sw_scania.tar.gz')
tf.extractall()

## 2. Hjälpmetoder och imports till NDVI och NDWI

För att alla delar i programmet ska fungera behöver vi importera biblioteken i blocket nedan. För att underlätta för er har vi också skrivit två hjälpfunktioner som reducerar arrayer. Detta innebär att de tar en array som är av storlek $h \times w$ och reducerar den till en array av storlek $\frac{h}{n} \times \frac{w}{n}$ där $n$ är parameter till funktionen. Vad detta innebär är att bilderna vi senare kollar på inte tar lika mycket plats som de annars skulle gjort, detta gör att våra beräkning går mycket snabbare. 

In [0]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import gc

def reduce_array(n, arr):
    r = arr.shape[0]
    c = arr.shape[1]
    out = np.empty(((r + n-1)//n, (c+n-1)//n))
    for i in range((r + n-1)//n):
        for j in range((c+n-1)//n):
            summa = 0
            num = 0
            for di in range(n):
                for dj in range(n):
                    if(i*n + di >= r or j*n + dj >= c):
                        continue
                    summa += arr[i*n+di][j*n+dj]
                    num += 1
            out[i][j] = summa/num
    return out

def reduce_array_2(n, arr):
    r = arr.shape[0]
    c = arr.shape[1]
    out = np.empty(((r + n-1)//n, (c+n-1)//n))
    for i in range((r + n-1)//n):
        for j in range((c+n-1)//n):
            if(i*n >= r or j*n >= c):
                continue
            out[i][j]=arr[i*n][j*n]
    return out

## 3. NDVI och dess VCI
Nu ska vi börja beräkna NDVI och VCI! (Vi förutsätter att du redan vet hur man beräknar NDVI, om du behöver pigga upp minnet, kolla på uppgiften torkan.) Nu ska vi kort förklara vad VCI är för något. VCI används för att jämföra ett NDVI för ett visst år med värden uppmätta de senate åren. VCI är en procentandel som visar var det nuvarande värdet befinner sig i förhållande till det största respektive minsta värdet under de undersökta åren. Ett lågt VCI indikerar på lågt NDVI jämfört med föregående år och därmed också ett en dålig växtlighet och ett tecken på torka.  Högt VCI tyder däremot på att att det är hög växtlighet jämfört med tidigare år och att det inte är torrt. VCI räknas ut genom 

<br>

$$VCI = \frac{NDVI_{\text{current}} - NDVI_{\text{min}}}{NDVI_{\text{max}} - NDVI_{\text{min}}} \times 100$$

<br>

för varje pixel. $NDVI_{\text{current}}$ är NDVI för året vi kollar på. $NDVI_{\text{min}}$ är det minsta NDVI-värdet för den pixeln under alla åren vi kollar på. Liknande är $NDVI_{\text{max}}$ det största NDVI-värdet  för den pixeln under alla åren vi kollar på. 

För att läsa in datan till programmet används funktionen nedan.

**Uppdrag:** Definiera funktionen genom att köra koden i blocket nedan. Läs sedan in datan i variabeln `raw_ndvi_data`. Vad betyder de olika stegen i funktionen? Vad får `raw_ndvi_data` för typ? Hur gör man för att komma åt det röda bandet från 2016? Hur många gånger har bilderna reducerats?

In [0]:
def get_raw_ndvi_data():
    ndvi_data = {}
    ndvi_data[2015] = {}
    ndvi_data[2015]["red"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2015/S2A_OPER_MSI_L1C_TL_EPA__20160721T205724_A000634_T33UUB_B04.jp2").ReadAsArray())
    ndvi_data[2015]["nir"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2015/S2A_OPER_MSI_L1C_TL_EPA__20160721T205724_A000634_T33UUB_B08.jp2").ReadAsArray())
    print("-----2015-done-----")
    ndvi_data[2016] = {}
    ndvi_data[2016]["red"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2016/S2A_OPER_MSI_L1C_TL_SGS__20160721T154816_A005639_T33UUB_B04.jp2").ReadAsArray())
    ndvi_data[2016]["nir"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2016/S2A_OPER_MSI_L1C_TL_SGS__20160721T154816_A005639_T33UUB_B08.jp2").ReadAsArray())
    print("-----2016-done-----")
    ndvi_data[2017] = {}
    ndvi_data[2017]["red"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2017/T33UUB_20170706T102021_B04.jp2").ReadAsArray())
    ndvi_data[2017]["nir"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2017/T33UUB_20170706T102021_B08.jp2").ReadAsArray())
    print("-----2017-done-----")
    ndvi_data[2018] = {}
    ndvi_data[2018]["red"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2018/T33UUB_20180726T102019_B04.jp2").ReadAsArray())
    ndvi_data[2018]["nir"] = reduce_array_2(10, gdal.Open("sentinel2_sw_scania/2018/T33UUB_20180726T102019_B08.jp2").ReadAsArray())
    print("-----2018-done-----")
    return ndvi_data

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>raw_ndvi_data = get_raw_ndvi_data()
</code></pre>
</details>

<details><summary markdown="span">Svar</summary>
<p>
    <code>raw_ndvi_data</code> är en <code>dict</code> med nycklar som är år. Värdet för varje år är ytterligare en <code>dict</code> med nycklarna <code>"red"</code> och <code>"nir"</code> som innehåller det röda och nära infraröda bandet. För att komma åt det röda bandet från 2016 skriver man därför <code>raw_ndvi_data[2016]["red"]</code>. Bilden har reducerats 10 gånger. Detta ser man eftersom det första agumentet till <code>reduce_array_2()</code> är 10. 
</details>

Nu ska vi beräkna NDVI precis som vi gjort innan. 

**Uppdrag:** Skapa en `dict` som heter `ndvi` som ska ha år som nycklar och innehålla NDVI för det året. Raden `np.seterr(divide='ignore', invalid='ignore')` gör att ni inte får problem om ni skulle råka dela med 0.

In [0]:
np.seterr(divide='ignore', invalid='ignore')


<details><summary markdown="span">Tips</summary>
<p>
    Gör en <code>for</code>-loop för att loopa igenom åren. 
    </p>   
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>ndvi = {}
for i in range(2015, 2019):
    ndvi[i] = (raw_ndvi_data[i]["nir"] - raw_ndvi_data[i]["red"])/(raw_ndvi_data[i]["nir"] + raw_ndvi_data[i]["red"])
</code></pre>
</details>

**Uppdrag:** Plotta NDVI för något år. Vilket ställe ser man på bilden?

<details><summary markdown="span">Tips</summary>
<p>
    Kolla hur du gjorde i förra uppgiften.
    </p>   
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>plt.figure(figsize=(10,10))
plt.imshow(ndvi[2018], aspect='auto', cmap='PiYG')
plt.clim(-1.0, 1.0)
plt.colorbar(label='NDVI ')
plt.show()
plt.savefig("ndvi.png")
plt.close()
</code></pre>
</details>

<details><summary markdown="span">Svar</summary>
<p>
   Man kan se sydvästra Skåne och en del av Danmark. 
    </p>   
</details>

Nu ska vi beräkna VCI.

**Uppdrag:** Skriv en funktion `calc_vci_ndvi(year)` som tar in ett år och beräknar VCI för detta året. Du ska returnera en array.

<details><summary markdown="span">Tips</summary>
<p>
    Använd antingen en <code>for</code>-loop för att loopa igenom alla pixlar eller så kan du använda <code>np.maximum(a, b)</code> och <code>np.minimum(a, b)</code> som returnerar en array där varje värde är det största respektive minsta värdet av <code>a</code> och <code>b</code> på den positionen.
    </p>   
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>def calc_vci_ndvi(year):
    max1 = np.maximum(ndvi[2015], ndvi[2016])
    max2 = np.maximum(ndvi[2017], ndvi[2018])
    max_total = np.maximum(max1, max2)
    min1 = np.minimum(ndvi[2015], ndvi[2016])
    min2 = np.minimum(ndvi[2017], ndvi[2018])
    min_total = np.minimum(min1, min2)
    vci = (ndvi[year] - min_total)/(max_total - min_total)
    return vci * 100
</code></pre>
</details>

Nu ska vi plotta VCI för att se hur mycket torka det är. 

**Uppdrag:** Skriv en funktion `plot_vci_ndvi(year)` som tar ett år och plottar VCI för det året. Använd denna funktionen för att plotta VCI för 2018. Ser det ut att vara torka?

<details><summary markdown="span">Tips</summary>
<p>
    Gör liknande som när du plottade NDVI. 
    </p>   
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>def plot_vci_ndvi(year):
    vci_ndvi = calc_vci_ndvi(year)
    plt.figure(figsize=(10,10))
    plt.imshow(vci_ndvi, aspect='auto', cmap='PiYG')
    plt.clim(0, 100)
    plt.colorbar(label='VCI-NDVI '+str(year))
    plt.show()
    plt.savefig("vci_ndvi" + str(year) + ".png")
    plt.close()

plot_vci_ndvi(2018)
</code></pre>
</details>

<details><summary markdown="span">Svar</summary>
<p>
   Det ser ganska lila ut, vilket betyder att 2018 har bland de lägre NDVI-värdena av de fyra åren. Detta tyder på torka.
    </p>   
</details>

## 4. NDWI och dess VCI
Nu ska vi kolla på det liknande NDWI istället för NDVI. De är ganska snarlika, men man kan säga att NDVI mäter vegetation medans NDWI snarare mäter fuktighet. Eftersom NDWI mäter fuktighet kan detta vara bra att kolla på för att undersöka torka. Vi ska nu gå igenom hur man kan beräkna NDWI.

Beräkningen är ganska lik NDVI men vi använder ett annat band. Vi använder fortfarande NIR men vi kommer också använda SWIR som står för Short-Wave Infrared band. Formeln är 

<br>

$$NDWI = \frac{(NIR-SWIR)}{(NIR+SWIR)}.$$

<br>

Även NDWI kommer alltid vara mellan -1och 1. Ett högt NDWI-värde indikerar på hög fuktighet, medans ett lågt indikerar på låg fuktighet. För ytterligare information om NDWI kan du läsa [här](https://en.wikipedia.org/wiki/Normalized_difference_water_index).

Vi kommer även kolla på VCI för NDWI. Som tur är funkar detta nästan exakt likadant. Formeln för VCI för NDWI är 

<br>

$$VCI = \frac{NDWI_{\text{current}} - NDWI_{\text{min}}}{NDWI_{\text{max}} - NDWI_{\text{min}}} \times 100.$$

<br>

Åter igen tyder ett lågt VCI på att detta året hade ovanlig lågt NDWI, medans ett högt tyder på att det hade ett högt NDWI.

**Uppdrag:** Kör följande kod för att definiera funktionen `get_raw_ndwi_data()` och spara datan i variabeln `raw_ndwi_data`. Vad är `raw_ndvi_data` för typ? Hur gör man för att komma åt NIR från 2016? Hur mycket reduceras bilden?

In [0]:
def get_raw_ndwi_data():
    ndwi_data = {}
    ndwi_data[2015] = {}
    ndwi_data[2015]["swir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2015/S2A_OPER_MSI_L1C_TL_EPA__20160721T205724_A000634_T33UUB_B11.jp2").ReadAsArray())
    ndwi_data[2015]["nir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2015/S2A_OPER_MSI_L1C_TL_EPA__20160721T205724_A000634_T33UUB_B8A.jp2").ReadAsArray())
    print("-----2015-done-----")
    ndwi_data[2016] = {}
    ndwi_data[2016]["swir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2016/S2A_OPER_MSI_L1C_TL_SGS__20160721T154816_A005639_T33UUB_B11.jp2").ReadAsArray())
    ndwi_data[2016]["nir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2016/S2A_OPER_MSI_L1C_TL_SGS__20160721T154816_A005639_T33UUB_B8A.jp2").ReadAsArray())
    print("-----2016-done-----")
    ndwi_data[2017] = {}
    ndwi_data[2017]["swir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2017/T33UUB_20170706T102021_B11.jp2").ReadAsArray())
    ndwi_data[2017]["nir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2017/T33UUB_20170706T102021_B8A.jp2").ReadAsArray())
    print("-----2017-done-----")
    ndwi_data[2018] = {}
    ndwi_data[2018]["swir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2018/T33UUB_20180726T102019_B11.jp2").ReadAsArray())
    ndwi_data[2018]["nir"] = reduce_array_2(5, gdal.Open("sentinel2_sw_scania/2018/T33UUB_20180726T102019_B8A.jp2").ReadAsArray())
    print("-----2018-done-----")
    return ndwi_data

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>raw_ndwi_data = get_raw_ndwi_data()
</code></pre>
</details>

<details><summary markdown="span">Svar</summary>
<p>
    <code>raw_ndwi_data</code> är en <code>dict</code> med nycklar som är år. Värdet för varje år är ytterligare en <code>dict</code> med nycklarna <code>"swir"</code> och <code>"nir"</code> som innehåller det kort våglängd infraröda bandet och nära infraröda bandet. För att komma åt det nära infraröda bandet från 2016 skriver man därför <code>raw_ndwi_data[2016]["nir"]</code>. Bilden har reducerats 5 gånger. Detta ser man eftersom det första agumentet till <code>reduce_array_2()</code> är 5. 
</details>

**Uppdrag:** Skapa nu en `dict` som heter `ndwi` som ska innehålla NDWI för varje år. 

In [0]:
np.seterr(divide='ignore', invalid='ignore')


<details><summary markdown="span">Tips</summary>
<p>
  Du kan återanvända mycket av koden du skrev innan för NDVI.
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>ndwi = {}
for i in range(2015, 2019):
    ndwi[i] = (raw_ndwi_data[i]["nir"] - raw_ndwi_data[i]["swir"])/(raw_ndwi_data[i]["nir"] + raw_ndwi_data[i]["swir"])
</code></pre>
</details>

**Uppdrag:** Gör nu liknande det du gjorde innan en funktion `calc_vci_ndwi(year)` och `plot_vci_ndwi(year)` fast för NDWI istället för NDVI. Använd detta för att plotta NDWI för 2018. Ser det torrt ut?

<details><summary markdown="span">Tips</summary>
<p>
    Du kan återanvända mycket av koden du använde för NDVI. 
    </p>   
</details>

<details><summary markdown="span">Lösning</summary>
<p>
<pre><code>def calc_vci_ndwi(year):
    max1 = np.maximum(ndwi[2015], ndwi[2016])
    max2 = np.maximum(ndwi[2017], ndwi[2018])
    max_total = np.maximum(max1, max2)
    min1 = np.minimum(ndwi[2015], ndwi[2016])
    min2 = np.minimum(ndwi[2017], ndwi[2018])
    min_total = np.minimum(min1, min2)
    vci = (ndwi[year] - min_total)/(max_total - min_total)
    return vci*100
    
def plot_vci_ndwi(year):
    vci_ndwi = calc_vci_ndwi(year)
    plt.figure(figsize=(10,10))
    plt.imshow(vci_ndwi, aspect='auto', cmap='PiYG')
    plt.clim(0, 100)
    plt.colorbar(label='VCI-NDWI '+str(year))
    plt.show()
    plt.savefig("vci_ndwi" + str(year) + ".png")
    plt.close()
    
plot_vci_ndwi(2018)
</code></pre>
</details>

<details><summary markdown="span">Svar</summary>
<p>
   Det ser ganska lila ut, vilket betyder att 2018 har bland de lägre NDWI-värdena av de fyra åren. Detta tyder på torka.
    </p>   
</details>

## Fortsättningsuppgifter
- Studera `reduce_array()` och `reduce_array_2()`. Hur fungerar dem, vad gör dem? Vilken är snabbare och varför? Testa att skriva en egen funktion som komprimerar arrayer. 